In [69]:
import pandas as pd

test_file = 'problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_2438_05022023_0001126908/text_formatted.txt'


formatted_output = pd.read_csv(test_file, sep='|')
formatted_output.head()



,Ricoh DCN,REGULATORY_APPROVAL_ID,Addressee,ADDRESS_LINE_1,ADDRESS_LINE_2,CITY,STATE,ZIP_CODE_4
0,C001244154.TIF,Y0020_WCM_100186E_C INTERNAL APPROVED 07252022...,PAUL LATREILLE,449 MOUNTAIN VIEW RD,NaN,WILLISTON,VT,05495-7723
1,C001244190.TIF,Y0020_WCM_100186E_C INTERNAL APPROVED 07252022...,PANAYOT KALOYANIDIS,10 PINE BROOK TER APT 1,NaN,BRISTOL,CT,06010-7409
2,C001244172.TIF,Y0020_WCM_100186E_C INTERNAL APPROVED 07252022...,ANNA J MONSCHEIN,PO BOX 404,NaN,COLUMBIANA,OH,44408-0404
3,C001244136.TIF,Y0020_WCM_100186E_C INTERNAL APPROVED 07252022...,SHARON C WAGNER,4316 HARVARD AVE,NaN,GREENSBORO,NC,27407-1712
4,C001244217.TIF,Y0020_WCM_100186E_C INTERNAL APPROVED 07252022...,JOANNE W WOODY,277 RIVERSIDE DR,NaN,CENTER CROSS,VA,22437-2008


In [70]:
import re

RICOH_DCN_COLUMN_NAME = "Ricoh DCN"
REGULATORY_APPROVAL_ID_COLUMN_NAME = "REGULATORY_APPROVAL_ID"
STATE_COLUMN_NAME = "STATE"
ZIP_CODE_4_COLUMN_NAME = "ZIP_CODE_4"


# We prepare the regular expressions from the lambda function extracting the text
def get_regular_expressions():
    """
    return: The dictionary having column names as keys and regular expressions (validating entries) as values
    Currently we validate REGULATORY_APPROVAL_ID, STATE, ZIP_CODE_4
    """
    approval_re = r"[(A-Z0-9_]+[_ ].*?(?:MATERIALS?|MODIFIED|MODIFIED_2023|ACCEPTED|SPN|)\d{7,8}|[A-Z0-9]+_[0-9A-Z]+_[A-Z0-9]+_[A-Z]+"
    twenty_re = (
        r"[A-Z][A-Z][OSC0-9][A-Z][A-Z][A-Z][A-Z0-9][A-Z0-9][A-Z0-9]+[ _.][A-Z\d]{3,4}"
    )
    field_regular_expressions = {
        # we allow the approval or twenty code missing
        REGULATORY_APPROVAL_ID_COLUMN_NAME: rf"^(({approval_re}) ({twenty_re})|({twenty_re})|({approval_re}))$",
        STATE_COLUMN_NAME: r"^[A-Z]{2}$",
        ZIP_CODE_4_COLUMN_NAME : "^[0-9]{5}(?:-[0-9]{4})?$",
    }

    # We compile all the regular expression in the above dictionary
    # overwriting the values
    for column in field_regular_expressions:
        field_regular_expressions[column] = re.compile(
            field_regular_expressions[column]
        )
    return field_regular_expressions


def restrict_to_wihout_reg_approval_id(field_regular_expressions):
    # restriction to the regular expressions without REGULATORY_APPROVAL_ID reg exp.
    field_regular_expressions_without_reg_id = {}

    for column in field_regular_expressions:
        if column != REGULATORY_APPROVAL_ID_COLUMN_NAME:
            field_regular_expressions_without_reg_id[
                column
            ] = field_regular_expressions[column]

    return field_regular_expressions_without_reg_id

In [71]:
import pandas as pd
from functools import reduce
from numpy import nan

DEFAULT_SEPARATOR = "|"
NA_PLACEHOLDER = "N/A"

CHECKED_COLUMN_SUFFIX = "_check"


field_regular_expressions = get_regular_expressions()
field_regular_expressions_without_reg_id = restrict_to_wihout_reg_approval_id(
    field_regular_expressions
)

NULL_SET = {
    "nan",
    "NaN",
    "None",
    "Null",
    "null",
    "NULL",
    "NIL",
    nan,
}


def get_filtered_formatted_output(formatted_output, field_regular_expressions):
    # Add f"{column}_check" columns to the formatted_output dataframe indicating if
    # a regular expression is violated by the column in a particular row
    for column in field_regular_expressions:
        formatted_output.loc[
            :, f"{column}{CHECKED_COLUMN_SUFFIX}"
        ] = formatted_output.loc[:, column].apply(
            lambda val: True
            if (
                not val
                or val in NULL_SET
                or not len(str(val))
                or field_regular_expressions[column].findall(str(val))
            )
            else False
        )

    # Filter only those row that have at least one violation of a regular expression in
    # a column.
    filtered_formatted_output = formatted_output[
        reduce(
            lambda acc, column: acc
            | (
                (formatted_output[f"{column}{CHECKED_COLUMN_SUFFIX}"] == False)
                & formatted_output[f"{column}"].notna()
                & formatted_output[f"{column}"].notnull()
            ),
            field_regular_expressions.keys(),
            False,
        )
    ]

    # Replace all the values in columns we are checking that weren't violating a regular expression in the
    # resulting filtered dataframe by "N/A".
    for column in field_regular_expressions:
        filtered_formatted_output.loc[:, f"{column}"] = filtered_formatted_output.loc[
            :, (f"{column}{CHECKED_COLUMN_SUFFIX}", column)
        ].apply(
            lambda row: row[column]
            if not row[f"{column}{CHECKED_COLUMN_SUFFIX}"]
            else NA_PLACEHOLDER,
            axis=1,
        )

    return filtered_formatted_output

In [72]:
def get_text_formatted_errors(
    batch,
    txt_file_in,
    txt_file_out_reg_id,
    txt_file_non_reg_id_out,
    separator=DEFAULT_SEPARATOR,
):
    """
    parses batch's txt_file_in file and saves the problems in txt_file_out_reg_id and txt_file_non_reg_id_out
    return:
    1) 4-tuple of the number of records (documents) in txt_file_in,
    2) number of documents with problems,
    3) dataframe containing rows having problems with REGULATORY_APPROVAL_ID,
    4) dataframe containing rows having problems with STATE or ZIP_CODE_4
    (a document can correspond to a row in both 3) and 4) and the value of 2) is
    the total number of documents appearing in the first or the second returned dataframe)
    """
    formatted_output = pd.read_csv(txt_file_in, sep=separator)

    filtered_formatted_output = get_filtered_formatted_output(
        formatted_output, field_regular_expressions
    )

    # documents with a problem in  REGULATORY_APPROVAL_ID
    filtered_reg_id_formatted_output = filtered_formatted_output[
        filtered_formatted_output[
            f"{REGULATORY_APPROVAL_ID_COLUMN_NAME}{CHECKED_COLUMN_SUFFIX}"
        ]
        == False
    ][[RICOH_DCN_COLUMN_NAME, REGULATORY_APPROVAL_ID_COLUMN_NAME]]

    # We only save if there is a record
    if filtered_reg_id_formatted_output.shape[0]:
        print(f"Saving: {txt_file_out_reg_id}")
        filtered_reg_id_formatted_output.to_csv(txt_file_out_reg_id, sep=separator)

    # documents with a problem in the STATE and ZIP_CODE_4
    filtered_without_reg_id_formatted_output = filtered_formatted_output[
        reduce(
            lambda acc, column: acc
            | (
                (filtered_formatted_output[f"{column}{CHECKED_COLUMN_SUFFIX}"] == False)
            ),
            field_regular_expressions_without_reg_id.keys(),
            False,
        )
    ]

    # restricting the output to the columns of interest
    filtered_without_reg_id_formatted_output = filtered_without_reg_id_formatted_output[
        [RICOH_DCN_COLUMN_NAME] + list(field_regular_expressions_without_reg_id.keys())
    ]
    # We only save if there is a record
    if filtered_without_reg_id_formatted_output.shape[0]:
        print(f"Saving: {txt_file_non_reg_id_out}")
        filtered_without_reg_id_formatted_output.to_csv(
            txt_file_non_reg_id_out, sep=separator
        )

    filtered_reg_id_formatted_output[RICOH_DCN_COLUMN_NAME] = (
        f"{batch}/" + filtered_reg_id_formatted_output[RICOH_DCN_COLUMN_NAME]
    )
    filtered_without_reg_id_formatted_output[RICOH_DCN_COLUMN_NAME] = (
        f"{batch}/" + filtered_without_reg_id_formatted_output[RICOH_DCN_COLUMN_NAME]
    )
    return (
        formatted_output.shape[0],
        filtered_formatted_output.shape[0],
        filtered_without_reg_id_formatted_output.astype(str),
        filtered_reg_id_formatted_output.astype(str),
    )

In [73]:
def clean_text_formatted_file(txt_file_in, txt_file_out, separator=DEFAULT_SEPARATOR):
    def clean_string(string_to_clean):
        while string_to_clean.find("  ") > -1:
            string_to_clean = string_to_clean.replace("  ", " ")
        return string_to_clean.strip().replace("\t", "").replace("\n", "")

    formatted_output = pd.read_csv(txt_file_in, sep="|")
    formatted_output = formatted_output.applymap(
        lambda value: clean_string(value) if isinstance(value, str) else value
    )
    formatted_output.to_csv(txt_file_out, sep=separator)

In [74]:
from pathlib import Path

paths = []

CLEANED_FILE_SUFFIX = "_cleaned.txt"
ERROR_NON_REG_ID_FILE_SUFFIX = "_non_reg_id_err.txt"
ERROR_REG_ID_FILE_SUFFIX = "_reg_id_err.txt"
TEXT_FORMATTED_FILENAME = "text_formatted.txt"
LAST_DOT_INDEX = -4


for filename in Path("problematic_samples/070723-zip/images-output").rglob(
    "text_formatted.txt"
):
    filename = str(filename)
    paths.append(
        (
            filename,
            f"{filename[:LAST_DOT_INDEX]}{CLEANED_FILE_SUFFIX}",
            f"{filename[:LAST_DOT_INDEX]}{ERROR_REG_ID_FILE_SUFFIX}",
            f"{filename[:LAST_DOT_INDEX]}{ERROR_NON_REG_ID_FILE_SUFFIX}",
        )
    )


output_non_reg_id_dcns = pd.DataFrame()
output_reg_id_dcns = pd.DataFrame()
total_row_count, total_bad_row_count = 0, 0
for path_in, cleaned_file, path_out_reg_id, path_out_non_reg_id in paths:
    batch = "/".join(path_in.split("/")[3:-1]).replace("delivery/", "")
    clean_text_formatted_file(path_in, cleaned_file)
    (
        row_count,
        bad_row_count,
        bad_non_reg_id_dcns,
        bad_reg_id_dcns,
    ) = get_text_formatted_errors(
        batch, cleaned_file, path_out_reg_id, path_out_non_reg_id
    )
    total_row_count += row_count
    total_bad_row_count += bad_row_count

    output_non_reg_id_dcns = pd.concat(
        (output_non_reg_id_dcns, bad_non_reg_id_dcns), axis=0
    )
    output_reg_id_dcns = pd.concat((output_reg_id_dcns, bad_reg_id_dcns), axis=0)

    # if bad_dcns.shape[0] > 0:
    #     bad_dcns_dict[batch] = list(bad_dcns.to_dict(orient="index").values())
    # if wrong_reg_id_dcns.shape[0] > 0:
    #     bad_reg_id_dict[batch] = list(
    #         wrong_reg_id_dcns.to_dict(orient="index").values()
    #     )


print(f"Total rows {total_row_count} and bad {total_bad_row_count}.")

output_non_reg_id_dcns.to_csv("output_non_reg_id_dcns.txt", index=False)
output_reg_id_dcns.to_csv("output_reg_id_dcns.txt", index=False)

# with open("check_regex_out.json", "w") as f:
#     f.write(json.dumps(output_non_reg_id_dcns))

# with open("check_reg_id_out.json", "w") as f:
#     f.write(json.dumps(output_reg_id_dcns))

Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_3211_05262023_0001141034/text_formatted_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_3211_05262023_0001141034/text_formatted_non_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_3168_05242023_0001139498/text_formatted_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_3309_05302023_0001142114/text_formatted_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_2901_05172023_0001135373/text_formatted_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_2820_05152023_0001134360/text_formatted_non_reg_id_err.txt
Saving: problematic_samples/070723-zip/images-output/07072023-2/delivery/CUR6470-01_3131_05232023_0001138687/text_formatted_reg_id_err.txt
Saving: problematic